In [ ]:
import os
import pandas as pd

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

a = %pwd # 현재 경로 a에 할당
os.chdir(a) # 파일 로드 경로 설정

# 데이터 로드 및 전처리

## 데이터 로드

In [ ]:
merged_df = pd.read_csv('상권별_리뷰.csv')

merged_df = pd.DataFrame() # 빈 데이터 프레임 생성


"""
DataFrame 통합
"""
for file in os.listdir("review_sample"): # file이 에너지 사용량 예측 폴더 내에 있는 파일명을 순회
    # csv확장자가 있는 경우만 불러오도록 
    df = pd.read_csv("review_sample/" + file) #파일들이 있는 폴더의 위치를 명시해준다.
    # 행 기준(axis=0)으로 빈 DataFrame과 불러온 DataFrame을 병합 + index는 새로 생성
    merged_df = pd.concat([merged_df, df], axis = 0, ignore_index = True)

"""
행단위로 붙일 경우 주로 ignore_index = True로 한다.
"""

In [ ]:
merged_df

## 전처리

### review내용 간단한 EDA

In [ ]:
# review 추출
corpus = merged_df['r_comments']
corpus

In [ ]:
# 데이터 전처리를 위한 전체 데이터의 특징을 빈도분석으로 파악
import nltk

# toal tokens 파악
total_tokens = [token for msg in corpus for token in str(msg).split()]
print(len(total_tokens))

In [ ]:
# 가장 많이 표현된 vocab TOP 10 추출
text = nltk.Text(total_tokens, name='NMSC')
print(len(set(text.tokens)))
print(text.vocab().most_common(10))

In [ ]:
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

plt.figure(figsize=(16, 10))
text.plot(50)

### 사용할 함수 지정 (text cleaning, tokenizer)

In [ ]:
# MeCab 함수 설정

"""
    Parsing 규칙의 문제점, split을 "," 기준으로 하는데, token이 "," 인 경우에는 쉼표만 잘려서 나오기 때문에, 
    + "%," 같이 특수문자와 쉼표가 같이 등장하는 경우도 생각해주어야 함.
    
    (",", "SC") 의 원래 튜플이 만들어지지 않음.
    
    명사 분석의 경우 해당 토큰이 필요하지 않으니 pass
    
    형태소 분석과 POS tagging의 경우 해당 토큰이 필요하므로, token[0]이 ' 인 경우엔 따로 (",", "SC")를 집어 넣어줘야함.
"""
import MeCab # 윈도우 명령어
import re

mecab = MeCab.Tagger()

def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR":
            nouns.append(token[0])
        
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    return pos

In [ ]:
import re

def message_cleaning(docs):

    """
        1. Photo, Emoticon은 내용알기 어려움 -> 제거
        
        2. 자음/모음 표현 처리방법.
            1) "ㅇㅇ" ,"ㅋㅋㅋㅋㅋ" 같은 자음만 존재하는 표현이나, "ㅡㅡ", "ㅠㅠ" 같은 모음만 존재하는 표현들은
            의미는 있으나 중요한 의미를 가지고 있지 않다고 판단하여 제거.
            
            2) 이러한 표현들도 전부 emoticon 같은 감정 표현의 의성어로 쓰거나, 단축 표현이므로 제거하지 않음. 
            
        3. http:// 로 시작하는 hyperlink 제거.
        
        4. 특수문자 제거.
    
    """
    # Series의 object를 str로 변경.
    docs = [str(doc) for doc in docs]
    
    # 1. 사진 & 이모티콘 제거 
    pattern1 = re.compile("사진|Emoticon")
    docs = [pattern1.sub("", doc) for doc in docs]
    
    # 2. 자음으로만 된 텍스트 & 모음으로만 된 텍스트 제거 (필요시)
    pattern2 = re.compile("[ㄱ-ㅎ]*[ㅏ-ㅢ]*")
    docs = [pattern2.sub("", doc) for doc in docs]
    
    # 3. hyperlink 제거 (from. googling)
    # http 로 시작하는 경우 : (https?:\/\/)
    # www.로 시작하는 경우 : ([\w.]+){1,2}
    # www 이후에 .com, .co.kr 등 : (\.[\w]{2,4}){1,2}(.*)
    pattern3 = re.compile(r"\b(https?:\/\/)?([\w.]+){1,2}(\.[\w]{2,4}){1,2}(.*)")
    docs = [pattern3.sub("", doc) for doc in docs]
    
    # 4 특수문자 제거
    pattern4 = re.compile("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")
    docs = [pattern4.sub("", doc) for doc in docs]

    return docs

# 불용어 추가
def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # SW.add("있다")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)
            
    return SW

# 토크나이저 
# mecab_nouns / mecab_morphs / mecab_pos 중 택 1
def text_tokenizing(doc):
    return [word for word in mecab_morphs(doc) if word not in SW and len(word) > 1]
    
    # wordcloud를 위해 명사만 추출하는 경우.
    #return [word for word in mecab.nouns(doc) if word not in SW and len(word) > 1]

### 리뷰 내용 클리닝

In [ ]:
SW = define_stopwords("stopwords-ko.txt") # 미리지정된 한국어 불용어 


# 카카오톡 텍스트를 정제 (특수문자, 자음&모음 제거, 링크제거, 특수문자 제거)
cleaned_corpus = message_cleaning(corpus)
print(len(cleaned_corpus)) # 정제된 텍스트 길이 확인
print(cleaned_corpus[:10])

In [ ]:
# 정제되어 지워지는 텍스트들을 확인하고 분석 대상에서 제외 해야한다. (내용이 없기 때문)

cleaned_text = pd.Series(cleaned_corpus) # 정제된 text
#merged_df["Message"] = cleaned_text # indexing으로 정제
cleaned_data = merged_df[merged_df["r_comments"] != ""] # 비어있는 str은 제외
cleaned_data.info() 

In [ ]:
# 필요 컬럼만 걸러내기
# 상권코드 or 상권코드명을 User로 사용
cleaned_data = cleaned_data[['r_date','TRDAR_CD','r_comments']]

#결과를 확인
cleaned_data.head()

In [ ]:
cleaned_data.info()

In [ ]:
# 정제한 리뷰를 pk로 저장

import pickle

with open("cleaned_review.pk", "wb") as f:
    pickle.dump(cleaned_data, f)

# LDA를 활용한 ATM( Author Topic Model)
---
- Author = 매장 , 매장별 토픽 추출 가능

In [ ]:
with open("cleaned_review.pk", "rb") as f:
    data = pickle.load(f)
    
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

## 상권별 데이터 추려내기

In [ ]:
# 사용자별로 데이터 처리를 해야 함
# 사용자 데이터 추려내기
users = set(data["TRDAR_CD"])
users

In [ ]:
from pprint import pprint
# User별로 groupby로 묶는다.
authors = data.groupby('TRDAR_CD')

# 결과 확인
pprint(authors.groups)
print(type(authors.groups))

In [ ]:
# groupby로 묶인 User 데이터를 Int64Index에서 list로 변경해준다.

author2doc = {} 

for user, index in authors.groups.items():
    author2doc[user] = list(index)
    
print(author2doc)

In [ ]:
# gensim에 들어갈 데이터를 생성
# LDA의 intput : [corpus, dictionary, topic개수] 등
# ATM도 마찬가지

# Message 컬럼의 각 row는 string -> 이것들이 tokenized된 것이 corpus
# gensim의 corpus와 dictionary를 만들려면 list of list of word 형태의 데이터가 있어야 한다.
# Message컬럼의 값을 list로 만들고, 하나 하나 불러와서 split() 해준다.
tokenized_data = [str(msg).split() for msg in list(data["r_comments"])]
print(tokenized_data[:10])

## gensim으로 ATM

In [ ]:
from gensim.models import AuthorTopicModel
from gensim.models import CoherenceModel
# bleicorpus : 결과 저장할때 쓰는 포멧
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

In [ ]:
"""
ATM에 사용할 Dictionary 만들기 (기본적으로 LDA와 동일)
corpus, dictionary는 데이터가 클수록 한번 만들때 오랜 시간이 소요 된다.
동일 데이터를 여러번 활용할것이라면 한번 만들었을때 저장해 두는 것이 좋다.
"""

# ATM에 사용할 Dictionary 만들기
# 해당 파일이 없으면 생성
if not os.path.exists('review(ATM)_dict'):
    # 토큰화한 데이터를 dictionary화 
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save('review(ATM)_dict') # 저장
    print(dictionary)
# 해당 파일이 존재하면 load
else:
    dictionary = Dictionary.load('review(ATM)_dict')

# ATM에 사용할 corpus 만들기
if not os.path.exists('review(ATM)_corpus'):
    # tokenized_data의 각 row를 BOW로 변형
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('review(ATM)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('review(ATM)_corpus')

In [ ]:
# ATM에 들어갈 데이터 확인
# 단어 개수, 문서개수, 저자의 수
print('매장 수: %d' % len(authors))
print('Unique한 토큰의 수: %d' % len(dictionary))
print('총 리뷰 수: %d' % len(corpus))

In [ ]:
# dictionary함수로 만든 사전에 있는 단어 보기
# 벡터화 한 곳에 텍스트가 들어갈 수 없으니 사전에 할당된 각 index값이 들어간다.
print(dictionary)
for idx in dictionary:
    print(dictionary[idx])

In [ ]:
# 사람이 이해할 수 있는 형태로 코퍼스 사전 재구성 (term-frequency) : 이 방식을 활용해 ATM모델링에 적용
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus]

## ATM 모델링

In [ ]:
# Author Topic Model 실행
if not os.path.exists("review(ATM)_model"):
    # 모델이 없으면 생성
    # num_topics는 default가 100 (5 개로 지정)
    # author2doc : 저자가 어떤 문서를 썼는가에 대한 index가 있는 dict
    model = AuthorTopicModel(corpus=corpus, num_topics=5, id2word=dictionary.id2token, \
                author2doc=author2doc, passes=10)
    model.save('review(ATM)_model')
else:
    model = AuthorTopicModel.load("review(ATM)_model")

In [ ]:
# 결과 확인
model.show_topic(0, topn=20) # 0번째 topic의 결과

## ATM  모델 평가

In [ ]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora
from tqdm import tqdm_notebook
from pprint import pprint

# 사용자간의 유사성을 평가하기 위한 measure를 사용하기 위해 불러오기
from gensim.matutils import hellinger
from gensim import matutils

import pandas as pd
import os

In [ ]:
NUM_TOPICS = 5

if not os.path.exists("review(ATM)_model"):
    model = AuthorTopicModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary.id2token, \
                author2doc=author2doc, passes=10)
    model.save('review(ATM)_model')
else:
    model = AuthorTopicModel.load("review(ATM)_model")

In [ ]:
# 토픽 라벨 지정.
topic_labels = ["Topic0", "Topic1", "Topic2", "Topic3", "Topic4"]

In [ ]:
# 토픽별로 topN 단어 확인하기.
for topic in model.show_topics(NUM_TOPICS):
    print('Label: ' + topic_labels[topic[0]])
    words = ''
    for word, prob in model.show_topic(topic[0], topn=20):
        words += word + ' '
    print('Words: ' + words)
    print()

In [ ]:
# 사용자별로 토픽 분포 확인하기.
def show_store(name):
    print('User: ',  name)
    print('Docs:', model.author2doc[name])
    print('Topics:')
    pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# "카페거실"의 토픽 분포 확인. (어느 토픽에 속할 확률이 가장 높은가)
# 추후 상권 단위 or 클러스터 로 묶는다면 단위별 토픽 분포 확인도 가능
show_store('카페거실')

In [ ]:
# Hellinger Distance를 개별로 구하기 위해 각 author(sotre)가 다른 authors간의 probability distributions를 도출
# 아래의 함수에 들어갈 예정
#[model.get_author_topics(author) for author in model.id2author.values()]

In [ ]:
# Hellinger Distance를 이용하여 비슷한 토픽을 가진 사용자(store)를 추정하는 함수.
# Hellinger Distance : 두개의  probability distributions 사이의 거리를 재는 방법
# store별 topic distribution이 들어간 df가 필요


# author-topic 분포 만들기
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]
 
def similarity(vec1, vec2):
    # vec1, vec2 사이의 hellinger similarity 구하기.
    dist = hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    # simulate : 1 / distance + 1  (smooth 형식)
    sim = 1.0 / (1.0 + dist)
    return sim
 
def get_sims(vec):
    # 각 사용자들 사이의 similarity pair 구하기.
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims
 
def get_table(name, top_n=10, smallest_author=1):
    """
    주어진 사용자에 대해서 topN 사람만큼 유사도를 
    정렬해서 table을 출력하는 함수
    top_n : 기본 10개
    smallest_author : review수가 지정된 값 이하이면 제외
    """
    
    # 유사도 측정하기
    # model.get_author_topics(name) : 각 author와 다른 authors 간의 distribution
    sims = get_sims(model.get_author_topics(name))
 
    # 저자별 정보 정렬하기
    # 비슷한 정도를 기준으로 정렬
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            
    # 사용자 패턴 분석 결과를 Dataframe으로 만들기
    # score = 유사도
    # size = review의 수
    df = pd.DataFrame(table, columns=['Store_Name', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n] # score순으로 정렬
    
    return df

In [ ]:
# 노원구 안에 있는 각 식당 = Author
# 제가 원래 하고 싶 었던 거는 -> 각 상권 =Author


# 10000개 -> 상권 코드 -> 코드별로 리뷰 수집 = corpus 
# 상권별 review -> ATM 분석 -> 유사상권 get

In [ ]:
# 매장별 리뷰에 대한 유사도 검증
get_table('제임스키친') # 상권A

- owner review : 각 상권에 owner review 가 어떤 영향을 미치는지
    - 상권 기준 owner revew /전체 리뷰 비율 -> 회귀 feature로 사용
    
y : 상권 매출 회귀 계수
x : 맛에 대한 표현 비율 , 분위기에 대한 표현 비율, 가성비에 대한 표현 비율, owner의 소통력(적극성지표), 평균 평점, 평균 리뷰수,

 1이하 평점수의 비율 => 1~2 는 부정 평가 -> 부정 평가 비율
3~5는 긍정평가  -> 긍정평가 비율

회귀 -> 영향력 평가